# 📊 Store Sales Time Series Forecasting

## 🎯 Competition Overview

In this competition, we will predict sales for thousands of product families sold at **Favorita stores** located in Ecuador. The training data includes dates, store and product information, promotional status, and sales numbers. Additional supplementary files are provided to enhance model building.

---

## 📑 Table of Contents

1. **Setup & Data Loading** — Import libraries, load all datasets
2. **Data Exploration & Quality** — Shape, types, missing values, duplicates
3. **Deep Exploratory Data Analysis (EDA)**
   - Overall sales trends (daily, monthly, yearly)
   - Sales by product family, store, city, state
   - Oil price analysis & correlation with sales
   - Holiday & event impact
   - Promotion impact analysis
   - Wage payment day effect (15th & last of month)
   - 2016 Ecuador earthquake impact
   - Time series decomposition
   - Day-of-week & monthly patterns
   - Transaction analysis
4. **Key Observations Summary** — Non-technical summary of findings
5. **Feature Engineering** — Building a modeling-ready dataset
6. **Modeling & Evaluation** — Multiple models compared
7. **Best Model & Final Predictions**

---

<div align="center">

[![LinkedIn](https://img.shields.io/badge/LinkedIn-Connect-0077B5?style=for-the-badge&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/sajjad-ali-shah47/)
[![Kaggle](https://img.shields.io/badge/Kaggle-Dataset-20BEFF?style=for-the-badge&logo=kaggle&logoColor=white)](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data)

---

**Author:** Sajjad Ali Shah | **Project Type:** Time Series Forecasting | **Tools:** Python, Pandas, Scikit-learn, LightGBM

</div>

---

## 1️⃣ Setup & Data Loading

Let's start by importing all necessary libraries and loading our datasets.

In [ ]:
# ============================================================
# 📦 Import Libraries
# ============================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from datetime import datetime, timedelta

# Time series
from statsmodels.tsa.seasonal import seasonal_decompose

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

# Settings
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 13

# Color palette
COLORS = sns.color_palette("husl", 12)
print("✅ All libraries imported successfully!")

In [ ]:
# ============================================================
# 📂 Load All Datasets
# ============================================================
DATA_PATH = 'Dataset/'

train = pd.read_csv(f'{DATA_PATH}train.csv', parse_dates=['date'])
test = pd.read_csv(f'{DATA_PATH}test.csv', parse_dates=['date'])
stores = pd.read_csv(f'{DATA_PATH}stores.csv')
oil = pd.read_csv(f'{DATA_PATH}oil.csv', parse_dates=['date'])
holidays = pd.read_csv(f'{DATA_PATH}holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv(f'{DATA_PATH}transactions.csv', parse_dates=['date'])
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

print("✅ All datasets loaded successfully!")
print(f"\n{'='*60}")
print("📊 Dataset Shapes:")
print(f"{'='*60}")
for name, df in [('Train', train), ('Test', test), ('Stores', stores),
                  ('Oil', oil), ('Holidays/Events', holidays),
                  ('Transactions', transactions), ('Sample Submission', submission)]:
    print(f"   • {name:20s}: {str(df.shape):>15s}")
print(f"{'='*60}")
print(f"\n📅 Training period: {train['date'].min().date()} to {train['date'].max().date()}")
print(f"📅 Test period:     {test['date'].min().date()} to {test['date'].max().date()}")
print(f"🏪 Number of stores: {train['store_nbr'].nunique()}")
print(f"📦 Product families:  {train['family'].nunique()}")

---

## 2️⃣ Data Exploration & Quality

Let's examine each dataset in detail — understanding data types, missing values, and basic statistics.

### 📈 Training Data

In [ ]:
# Training data overview
print("=" * 70)
print("📊 TRAINING DATA")
print("=" * 70)
display(train.head(10))
print(f"\nShape: {train.shape}")
print(f"\nData Types:\n{train.dtypes}")
print(f"\nMissing Values:\n{train.isnull().sum()}")
print(f"\nBasic Statistics:")
display(train.describe())
print(f"\n📦 Product Families ({train['family'].nunique()}):")
print(train['family'].unique())

### 🏪 Stores Data

In [ ]:
# Stores overview
print("=" * 70)
print("🏪 STORES DATA")
print("=" * 70)
display(stores)
print(f"\nCities: {stores['city'].nunique()} | States: {stores['state'].nunique()}")
print(f"Store Types: {stores['type'].unique()}")
print(f"Clusters: {sorted(stores['cluster'].unique())}")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
stores['type'].value_counts().plot(kind='bar', ax=axes[0], color=COLORS[:5], edgecolor='black')
axes[0].set_title('🏪 Stores by Type', fontweight='bold')
axes[0].set_ylabel('Count')

stores['state'].value_counts().plot(kind='barh', ax=axes[1], color=COLORS[2], edgecolor='black')
axes[1].set_title('🗺️ Stores by State', fontweight='bold')

stores['cluster'].value_counts().sort_index().plot(kind='bar', ax=axes[2], color=COLORS[4], edgecolor='black')
axes[2].set_title('📊 Stores by Cluster', fontweight='bold')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.show()

### 🛢️ Oil Price Data

In [ ]:
# Oil data overview
print("=" * 70)
print("🛢️ OIL PRICE DATA")
print("=" * 70)
print(f"Shape: {oil.shape}")
print(f"Date range: {oil['date'].min().date()} to {oil['date'].max().date()}")
print(f"\nMissing values: {oil.isnull().sum().to_dict()}")
print(f"Missing oil prices: {oil['dcoilwtico'].isnull().sum()} ({oil['dcoilwtico'].isnull().mean()*100:.1f}%)")

# Fill missing oil prices with interpolation
oil['dcoilwtico'] = oil['dcoilwtico'].interpolate(method='linear')
oil['dcoilwtico'] = oil['dcoilwtico'].bfill()
print(f"\n✅ Oil prices interpolated. Remaining NaN: {oil['dcoilwtico'].isnull().sum()}")

display(oil.describe())

### 🎉 Holidays & Events Data

In [ ]:
# Holidays overview
print("=" * 70)
print("🎉 HOLIDAYS & EVENTS DATA")
print("=" * 70)
display(holidays.head(10))
print(f"\nShape: {holidays.shape}")
print(f"\nHoliday Types:\n{holidays['type'].value_counts()}")
print(f"\nLocale Types:\n{holidays['locale'].value_counts()}")
print(f"\nTransferred: {holidays['transferred'].value_counts().to_dict()}")

### 🧾 Transactions Data

In [ ]:
# Transactions overview
print("=" * 70)
print("🧾 TRANSACTIONS DATA")
print("=" * 70)
display(transactions.head(10))
print(f"\nShape: {transactions.shape}")
print(f"Date range: {transactions['date'].min().date()} to {transactions['date'].max().date()}")
display(transactions.describe())

---

## 3️⃣ Deep Exploratory Data Analysis (EDA)

Now let's dive deep into the data with professional visualizations. Each chart includes annotations and observations to help **non-technical readers** understand the patterns.

---

### 📈 3.1 Overall Sales Trends

> **What are we looking at?** Total daily sales across ALL stores over time. This helps us see the big picture — growth trends, seasonality, and major disruptions.

In [ ]:
# Daily total sales
daily_sales = train.groupby('date')['sales'].sum().reset_index()

fig, axes = plt.subplots(3, 1, figsize=(16, 14))

# Daily
axes[0].plot(daily_sales['date'], daily_sales['sales'], linewidth=0.5, alpha=0.7, color=COLORS[0])
axes[0].set_title('📈 Daily Total Sales (All Stores)', fontweight='bold', fontsize=16)
axes[0].set_ylabel('Total Sales')
axes[0].axvline(pd.Timestamp('2016-04-16'), color='red', linestyle='--', alpha=0.8, label='🌍 Earthquake (Apr 16, 2016)')
axes[0].legend(fontsize=11)

# Monthly
monthly_sales = train.groupby(train['date'].dt.to_period('M'))['sales'].sum().reset_index()
monthly_sales['date'] = monthly_sales['date'].dt.to_timestamp()
axes[1].plot(monthly_sales['date'], monthly_sales['sales'], linewidth=2, color=COLORS[1], marker='o', markersize=3)
axes[1].set_title('📅 Monthly Total Sales', fontweight='bold', fontsize=16)
axes[1].set_ylabel('Total Sales')
axes[1].axvline(pd.Timestamp('2016-04-01'), color='red', linestyle='--', alpha=0.8, label='🌍 Earthquake Month')
axes[1].legend(fontsize=11)

# Yearly
yearly_sales = train.groupby(train['date'].dt.year)['sales'].sum().reset_index()
yearly_sales.columns = ['year', 'sales']
axes[2].bar(yearly_sales['year'].astype(str), yearly_sales['sales'], color=COLORS[2], edgecolor='black', alpha=0.8)
axes[2].set_title('📊 Yearly Total Sales', fontweight='bold', fontsize=16)
axes[2].set_ylabel('Total Sales')
for i, v in enumerate(yearly_sales['sales']):
    axes[2].text(i, v + v*0.01, f'{v/1e6:.1f}M', ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.show()

**🔍 Observation:** Sales show a clear **upward trend** over the years, with strong **seasonal spikes** (likely around holidays like Christmas). The 2016 earthquake caused a visible disruption in the daily sales pattern.

---

### 📦 3.2 Sales by Product Family

> **What are we looking at?** Which product categories sell the most? This helps the business decide where to focus inventory and promotions.

In [ ]:
# Top and Bottom product families
family_sales = train.groupby('family')['sales'].sum().sort_values(ascending=False).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top 10
top10 = family_sales.head(10)
bars1 = axes[0].barh(top10['family'], top10['sales'], color=sns.color_palette('YlOrRd_r', 10), edgecolor='black')
axes[0].set_title('🏆 Top 10 Product Families by Sales', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Total Sales')
axes[0].invert_yaxis()
for bar, val in zip(bars1, top10['sales']):
    axes[0].text(val + val*0.01, bar.get_y() + bar.get_height()/2,
                f'{val/1e6:.1f}M', va='center', fontweight='bold', fontsize=9)

# Bottom 10
bot10 = family_sales.tail(10)
bars2 = axes[1].barh(bot10['family'], bot10['sales'], color=sns.color_palette('Blues_r', 10), edgecolor='black')
axes[1].set_title('📉 Bottom 10 Product Families by Sales', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Total Sales')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

**🔍 Observation:** **GROCERY I** and **BEVERAGES** dominate sales massively, accounting for a significant share. Some categories like **BOOKS** and **BABY CARE** have very low sales volumes — these may be niche products carried in select stores.

---

### 🏪 3.3 Sales by Store, City & State

> **What are we looking at?** Which stores and locations generate the most revenue?

In [ ]:
# Merge store info with sales
train_stores = train.merge(stores, on='store_nbr', how='left')

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Sales by store type
type_sales = train_stores.groupby('type')['sales'].mean().sort_values(ascending=False)
type_sales.plot(kind='bar', ax=axes[0,0], color=COLORS[:5], edgecolor='black', alpha=0.85)
axes[0,0].set_title('🏪 Average Daily Sales by Store Type', fontweight='bold')
axes[0,0].set_ylabel('Average Sales')
axes[0,0].tick_params(axis='x', rotation=0)

# Sales by state (top 10)
state_sales = train_stores.groupby('state')['sales'].sum().sort_values(ascending=False).head(10)
state_sales.plot(kind='barh', ax=axes[0,1], color=COLORS[3], edgecolor='black', alpha=0.85)
axes[0,1].set_title('🗺️ Top 10 States by Total Sales', fontweight='bold')
axes[0,1].invert_yaxis()

# Sales by city (top 10)
city_sales = train_stores.groupby('city')['sales'].sum().sort_values(ascending=False).head(10)
city_sales.plot(kind='barh', ax=axes[1,0], color=COLORS[5], edgecolor='black', alpha=0.85)
axes[1,0].set_title('🏙️ Top 10 Cities by Total Sales', fontweight='bold')
axes[1,0].invert_yaxis()

# Sales by cluster
cluster_sales = train_stores.groupby('cluster')['sales'].mean().sort_values(ascending=False)
cluster_sales.plot(kind='bar', ax=axes[1,1], color=COLORS[7], edgecolor='black', alpha=0.85)
axes[1,1].set_title('📊 Average Daily Sales by Cluster', fontweight='bold')
axes[1,1].set_ylabel('Average Sales')

plt.tight_layout()
plt.show()

**🔍 Observation:** Stores of **Type D** tend to have the highest average daily sales. **Pichincha** (the state containing the capital Quito) leads state-level sales. Quito itself is the top city. Store performance varies significantly by cluster.

---

### 🛢️ 3.4 Oil Price Analysis

> **Why does oil matter?** Ecuador is an oil-dependent country. When oil prices drop, the economy suffers, and supermarket sales may be affected.

In [ ]:
# Merge oil with daily sales
daily_sales_oil = daily_sales.merge(oil, on='date', how='left')
daily_sales_oil['dcoilwtico'] = daily_sales_oil['dcoilwtico'].interpolate().bfill()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Oil price over time
axes[0].plot(daily_sales_oil['date'], daily_sales_oil['dcoilwtico'], color='darkgreen', linewidth=1.5)
axes[0].set_title('🛢️ Daily Oil Price (WTI Crude)', fontweight='bold', fontsize=16)
axes[0].set_ylabel('Price (USD/barrel)')
axes[0].fill_between(daily_sales_oil['date'], daily_sales_oil['dcoilwtico'], alpha=0.2, color='green')

# Scatter: Oil vs Sales
axes[1].scatter(daily_sales_oil['dcoilwtico'], daily_sales_oil['sales'],
               alpha=0.15, s=8, color=COLORS[0])
axes[1].set_title('🔗 Oil Price vs Total Daily Sales', fontweight='bold', fontsize=16)
axes[1].set_xlabel('Oil Price (USD/barrel)')
axes[1].set_ylabel('Total Daily Sales')

# Correlation
corr = daily_sales_oil[['dcoilwtico', 'sales']].corr().iloc[0,1]
axes[1].text(0.05, 0.95, f'Correlation: {corr:.3f}', transform=axes[1].transAxes,
            fontsize=14, fontweight='bold', va='top',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

**🔍 Observation:** Oil prices show a dramatic decline from ~$100 in 2014 to below $30 in early 2016, followed by a recovery. The correlation between oil prices and sales appears weak, suggesting that while oil impacts the economy broadly, supermarket sales depend more on local factors like promotions, holidays, and seasons.

---

### 🎉 3.5 Holiday & Event Impact on Sales

> **Do holidays boost or reduce sales?** Some holidays close stores (zero sales), while events like Christmas drive massive spending.

In [ ]:
# Create holiday flag for training data
national_holidays = holidays[
    (holidays['locale'] == 'National') &
    (holidays['transferred'] == False) &
    (holidays['type'].isin(['Holiday', 'Additional', 'Transfer']))
]['date'].unique()

daily_sales['is_holiday'] = daily_sales['date'].isin(national_holidays)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Average sales: holiday vs non-holiday
holiday_avg = daily_sales.groupby('is_holiday')['sales'].mean()
bars = axes[0].bar(['Regular Day', 'National Holiday'], holiday_avg.values,
                   color=[COLORS[0], COLORS[3]], edgecolor='black', alpha=0.85, width=0.5)
axes[0].set_title('🎉 Average Daily Sales: Holiday vs Regular', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Average Total Sales')
for bar, val in zip(bars, holiday_avg.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, val + val*0.01,
                f'{val:,.0f}', ha='center', fontweight='bold', fontsize=12)

# Sales by holiday type
holiday_dates = holidays[holidays['type'].isin(['Holiday', 'Additional', 'Transfer', 'Bridge', 'Event'])].copy()
daily_with_htype = daily_sales.merge(holiday_dates[['date', 'type']].drop_duplicates('date'),
                                      on='date', how='left')
daily_with_htype['type'] = daily_with_htype['type'].fillna('Regular')
htype_avg = daily_with_htype.groupby('type')['sales'].mean().sort_values(ascending=False)

htype_avg.plot(kind='bar', ax=axes[1], color=sns.color_palette('Set2', len(htype_avg)),
              edgecolor='black', alpha=0.85)
axes[1].set_title('📊 Average Sales by Day Type', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Average Total Sales')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

**🔍 Observation:** National holidays actually show **lower average sales** compared to regular days, likely because many stores close or operate with reduced hours. **Events** and **Transfer** days (when a holiday celebration is moved) tend to have higher sales — people prepare for celebrations!

---

### 🎯 3.6 Promotion Impact on Sales

> **Do promotions work?** We compare average sales for items on promotion vs not on promotion.

In [ ]:
# Promotion impact by family
train['has_promo'] = train['onpromotion'] > 0
promo_impact = train.groupby(['family', 'has_promo'])['sales'].mean().unstack()
promo_impact.columns = ['No Promo', 'With Promo']
promo_impact['Lift'] = ((promo_impact['With Promo'] - promo_impact['No Promo']) /
                         promo_impact['No Promo'] * 100)
promo_impact = promo_impact.sort_values('Lift', ascending=False)

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Overall promo effect
overall = train.groupby('has_promo')['sales'].mean()
bars = axes[0].bar(['No Promotion', 'On Promotion'], overall.values,
                   color=[COLORS[0], COLORS[4]], edgecolor='black', alpha=0.85, width=0.5)
axes[0].set_title('🎯 Average Sales: Promo vs No Promo', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Average Sales per Record')
for bar, val in zip(bars, overall.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, val + val*0.02,
                f'{val:.1f}', ha='center', fontweight='bold', fontsize=12)

# Top 15 families by promo lift
top_lift = promo_impact.head(15)
top_lift['Lift'].plot(kind='barh', ax=axes[1], color=COLORS[4], edgecolor='black', alpha=0.85)
axes[1].set_title('📊 Top 15 Families by Promotion Sales Lift (%)', fontweight='bold', fontsize=14)
axes[1].set_xlabel('Sales Lift (%)')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

**🔍 Observation:** Promotions significantly boost sales! Items on promotion sell substantially more on average. Some product families see **massive lifts** from promotions, making them ideal candidates for promotional campaigns.

---

### 💰 3.7 Wage Payment Day Effect

> **When do people get paid?** Public sector wages in Ecuador are paid on the **15th** and the **last day of the month**. We expect sales spikes on these days.

In [ ]:
# Payday analysis
daily_sales['day'] = daily_sales['date'].dt.day
daily_sales['is_month_end'] = daily_sales['date'].dt.is_month_end
daily_sales['is_15th'] = daily_sales['day'] == 15
daily_sales['is_payday'] = daily_sales['is_15th'] | daily_sales['is_month_end']

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Payday vs non-payday
payday_avg = daily_sales.groupby('is_payday')['sales'].mean()
bars = axes[0].bar(['Regular Day', 'Pay Day (15th/EOM)'], payday_avg.values,
                   color=[COLORS[0], COLORS[6]], edgecolor='black', alpha=0.85, width=0.5)
axes[0].set_title('💰 Average Sales: Pay Day vs Regular Day', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Average Total Sales')
for bar, val in zip(bars, payday_avg.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, val + val*0.01,
                f'{val:,.0f}', ha='center', fontweight='bold', fontsize=12)

# Sales by day of month
day_avg = daily_sales.groupby('day')['sales'].mean()
colors_day = ['red' if d in [15, 28, 29, 30, 31] else COLORS[0] for d in day_avg.index]
day_avg.plot(kind='bar', ax=axes[1], color=colors_day, edgecolor='black', alpha=0.8)
axes[1].set_title('📅 Average Sales by Day of Month', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Average Total Sales')
axes[1].tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

**🔍 Observation:** Pay days show **higher average sales** than regular days. The pattern is visible in the day-of-month chart where the **15th and end-of-month days** (colored red) tend to spike. People shop more right after getting paid!

---

### 🌍 3.8 Impact of the 2016 Ecuador Earthquake

> **On April 16, 2016,** a magnitude 7.8 earthquake struck Ecuador. Relief efforts led to mass donations and affected supermarket sales for weeks.

In [ ]:
# Earthquake analysis
eq_date = pd.Timestamp('2016-04-16')
eq_window = daily_sales[
    (daily_sales['date'] >= eq_date - timedelta(days=60)) &
    (daily_sales['date'] <= eq_date + timedelta(days=60))
].copy()

fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(eq_window['date'], eq_window['sales'], linewidth=2, color=COLORS[0], marker='o', markersize=3)
ax.axvline(eq_date, color='red', linestyle='--', linewidth=2, label='🌍 Earthquake (Apr 16)')
ax.axvspan(eq_date, eq_date + timedelta(days=14), alpha=0.15, color='red', label='2 weeks after')
ax.set_title('🌍 Sales Around the 2016 Ecuador Earthquake (±60 days)', fontweight='bold', fontsize=16)
ax.set_ylabel('Total Daily Sales')
ax.legend(fontsize=12)
plt.tight_layout()
plt.show()

# Before vs After comparison
before = daily_sales[(daily_sales['date'] >= eq_date - timedelta(days=30)) &
                     (daily_sales['date'] < eq_date)]['sales'].mean()
during = daily_sales[(daily_sales['date'] >= eq_date) &
                     (daily_sales['date'] < eq_date + timedelta(days=14))]['sales'].mean()
after = daily_sales[(daily_sales['date'] >= eq_date + timedelta(days=14)) &
                    (daily_sales['date'] <= eq_date + timedelta(days=60))]['sales'].mean()

print(f"📊 Average daily sales comparison:")
print(f"   Before earthquake (30 days): {before:>12,.0f}")
print(f"   During/relief (14 days):     {during:>12,.0f}")
print(f"   After (14-60 days):          {after:>12,.0f}")
print(f"   Change during relief:        {((during-before)/before*100):>+10.1f}%")

**🔍 Observation:** The earthquake had a dramatic effect on sales. In the immediate aftermath, sales patterns changed significantly as relief efforts and emergency buying altered consumer behavior. Sales eventually recovered as the situation stabilized.

---

### 🔬 3.9 Time Series Decomposition

> **Breaking down the signal:** We decompose the sales time series into **Trend** (long-term direction), **Seasonality** (repeating patterns), and **Residual** (random noise).

In [ ]:
# Use weekly aggregation for cleaner decomposition
weekly_sales = daily_sales.set_index('date')['sales'].resample('W').sum()

decomp = seasonal_decompose(weekly_sales, model='multiplicative', period=52)

fig, axes = plt.subplots(4, 1, figsize=(16, 14))
components = [('Observed', decomp.observed), ('Trend', decomp.trend),
              ('Seasonal', decomp.seasonal), ('Residual', decomp.resid)]
comp_colors = [COLORS[0], COLORS[1], COLORS[3], COLORS[5]]

for ax, (name, data), color in zip(axes, components, comp_colors):
    ax.plot(data, color=color, linewidth=1.5)
    ax.set_title(f'📊 {name}', fontweight='bold', fontsize=14)
    ax.set_ylabel(name)

plt.tight_layout()
plt.show()

**🔍 Observation:** The decomposition reveals: (1) A clear **upward trend** in sales over the years, (2) Strong **yearly seasonality** with peaks around December (holiday season), and (3) The residual shows the earthquake disruption as an anomaly.

---

### 📅 3.10 Day-of-Week & Monthly Patterns

> **When do people shop most?** These cyclical patterns are crucial for accurate forecasting.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Day of week
daily_sales['day_of_week'] = daily_sales['date'].dt.day_name()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_avg = daily_sales.groupby('day_of_week')['sales'].mean().reindex(dow_order)
dow_colors = [COLORS[i] for i in range(7)]
dow_avg.plot(kind='bar', ax=axes[0], color=dow_colors, edgecolor='black', alpha=0.85)
axes[0].set_title('📅 Average Sales by Day of Week', fontweight='bold', fontsize=14)
axes[0].set_ylabel('Average Total Sales')
axes[0].tick_params(axis='x', rotation=45)

# Monthly pattern
daily_sales['month'] = daily_sales['date'].dt.month_name()
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
month_avg = daily_sales.groupby('month')['sales'].mean().reindex(month_order)
month_avg.plot(kind='bar', ax=axes[1], color=sns.color_palette('coolwarm', 12), edgecolor='black', alpha=0.85)
axes[1].set_title('📅 Average Sales by Month', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Average Total Sales')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

**🔍 Observation:** **Sundays** are the busiest shopping day, while sales tend to be lower mid-week. **December** has the highest monthly average (holiday shopping), and there's also a visible spike in **April** (possibly Easter-related).

---

### 🧾 3.11 Transaction Analysis

> **How do transactions correlate with sales?** More transactions should mean more sales.

In [ ]:
# Merge transactions with daily store sales
store_daily = train.groupby(['date', 'store_nbr'])['sales'].sum().reset_index()
store_trans = store_daily.merge(transactions, on=['date', 'store_nbr'], how='inner')

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter: transactions vs sales
axes[0].scatter(store_trans['transactions'], store_trans['sales'],
               alpha=0.05, s=5, color=COLORS[0])
corr_ts = store_trans[['transactions', 'sales']].corr().iloc[0,1]
axes[0].set_title(f'🔗 Transactions vs Sales (corr={corr_ts:.3f})', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Daily Transactions')
axes[0].set_ylabel('Daily Sales')

# Average transactions by day of week
trans_dow = transactions.copy()
trans_dow['dow'] = trans_dow['date'].dt.day_name()
dow_trans = trans_dow.groupby('dow')['transactions'].mean().reindex(dow_order)
dow_trans.plot(kind='bar', ax=axes[1], color=dow_colors, edgecolor='black', alpha=0.85)
axes[1].set_title('🧾 Average Transactions by Day of Week', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Average Transactions')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

**🔍 Observation:** There is a **strong positive correlation** between transactions and sales — more customers visiting means more revenue. The day-of-week pattern for transactions mirrors the sales pattern, confirming that **Sunday** is the busiest day.

---

---

## 4️⃣ Key Observations Summary

Here is a **plain-language summary** of everything we discovered — written so anyone can understand it, even without a technical background.

---

### 📊 What Drives Sales?

| Factor | Impact | Details |
|--------|--------|---------|
| **Product Family** | 🔴 Very High | GROCERY I and BEVERAGES account for the lion's share of sales |
| **Promotions** | 🔴 Very High | Items on promotion sell significantly more — promotions work! |
| **Day of Week** | 🟡 Medium | Sunday is the busiest; mid-week is slower |
| **Pay Days** | 🟡 Medium | Sales spike on the 15th and last day of month (payday) |
| **Holidays** | 🟡 Medium | Holidays can reduce sales (store closures) but events increase them |
| **Season/Month** | 🟡 Medium | December (Christmas) is the peak month |
| **Store Type/Location** | 🟡 Medium | Type D stores and Quito perform best |
| **Transactions** | 🔴 Very High | Strong correlation — more visitors = more sales |

---

### 🛢️ Oil Price & the Economy
- Oil prices show major fluctuations but have a **weak direct correlation** with daily sales
- The impact is more **indirect** — through the broader economy, employment, and consumer confidence
- Oil price crashed from $100 to below $30 during 2014-2016, coinciding with slower sales growth

### 🌍 The 2016 Earthquake
- The April 16, 2016 earthquake **disrupted normal sales patterns** for several weeks
- Immediate aftermath showed unusual buying patterns (relief supplies)
- Sales eventually normalized but the event is a clear outlier in the data

### 📈 Long-Term Trend
- Sales show a **consistent upward trend** from 2013 to 2017
- Strong **yearly seasonality** with December peaks
- The business is growing overall, which is a positive sign

---

---

## 5️⃣ Feature Engineering

Now we build a **modeling-ready dataset** by creating meaningful features from our raw data. Good features are the key to accurate forecasting!

In [ ]:
# ============================================================
# 🔧 Feature Engineering
# ============================================================

# Start with train data
df = train.copy()

# ---------- Date Features ----------
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek
df['day_of_year'] = df['date'].dt.dayofyear
df['week_of_year'] = df['date'].dt.isocalendar().week.astype(int)
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_month_start'] = df['date'].dt.is_month_start.astype(int)
df['is_month_end'] = df['date'].dt.is_month_end.astype(int)
df['is_payday'] = ((df['day'] == 15) | df['date'].dt.is_month_end).astype(int)

# Earthquake flag
eq_start = pd.Timestamp('2016-04-16')
eq_end = pd.Timestamp('2016-05-15')
df['is_earthquake_period'] = ((df['date'] >= eq_start) & (df['date'] <= eq_end)).astype(int)

print("✅ Date features created")
print(f"   Features: year, month, day, day_of_week, day_of_year, week_of_year")
print(f"   Flags: is_weekend, is_month_start, is_month_end, is_payday, is_earthquake_period")

In [ ]:
# ---------- Merge Store Info ----------
df = df.merge(stores, on='store_nbr', how='left')
print(f"✅ Store info merged: city, state, type, cluster")

# ---------- Merge Oil Prices ----------
df = df.merge(oil, on='date', how='left')
df['dcoilwtico'] = df['dcoilwtico'].interpolate(method='linear').bfill().ffill()
print(f"✅ Oil prices merged and interpolated")

# ---------- Holiday Features ----------
# Create a simplified holiday indicator
nat_holidays = holidays[
    (holidays['locale'] == 'National') &
    (holidays['type'].isin(['Holiday', 'Additional', 'Transfer']))
]
df['is_national_holiday'] = df['date'].isin(nat_holidays['date'].unique()).astype(int)

events = holidays[holidays['type'] == 'Event']
df['is_event'] = df['date'].isin(events['date'].unique()).astype(int)

print(f"✅ Holiday features created: is_national_holiday, is_event")

In [ ]:
# ---------- Encode Categorical Variables ----------
le_family = LabelEncoder()
le_city = LabelEncoder()
le_state = LabelEncoder()
le_type = LabelEncoder()

df['family_encoded'] = le_family.fit_transform(df['family'])
df['city_encoded'] = le_city.fit_transform(df['city'])
df['state_encoded'] = le_state.fit_transform(df['state'])
df['type_encoded'] = le_type.fit_transform(df['type'])

print(f"✅ Categorical variables encoded")
print(f"   Families: {df['family_encoded'].nunique()}, Cities: {df['city_encoded'].nunique()}")
print(f"   States: {df['state_encoded'].nunique()}, Types: {df['type_encoded'].nunique()}")

In [ ]:
# ---------- Lag & Rolling Features (aggregated by store+family) ----------
# Sort for proper lag computation
df = df.sort_values(['store_nbr', 'family', 'date']).reset_index(drop=True)

# Create lags and rolling features per store-family combination
for lag in [7, 14, 28]:
    df[f'sales_lag_{lag}'] = df.groupby(['store_nbr', 'family_encoded'])['sales'].shift(lag)

for window in [7, 14, 28]:
    df[f'sales_roll_mean_{window}'] = (
        df.groupby(['store_nbr', 'family_encoded'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).mean())
    )
    df[f'sales_roll_std_{window}'] = (
        df.groupby(['store_nbr', 'family_encoded'])['sales']
        .transform(lambda x: x.shift(1).rolling(window, min_periods=1).std())
    )

print(f"✅ Lag features created: sales_lag_7, sales_lag_14, sales_lag_28")
print(f"✅ Rolling features created: mean and std for windows 7, 14, 28")

In [ ]:
# ---------- Drop rows with NaN from lag features ----------
print(f"\nBefore dropping NaN: {df.shape}")
df_model = df.dropna().copy()
print(f"After dropping NaN:  {df_model.shape}")
print(f"Dropped: {len(df) - len(df_model):,} rows ({(len(df)-len(df_model))/len(df)*100:.1f}%)")

# Final feature columns
feature_cols = [
    'store_nbr', 'family_encoded', 'onpromotion',
    'year', 'month', 'day', 'day_of_week', 'day_of_year', 'week_of_year',
    'is_weekend', 'is_month_start', 'is_month_end', 'is_payday',
    'is_earthquake_period', 'is_national_holiday', 'is_event',
    'cluster', 'type_encoded', 'city_encoded', 'state_encoded',
    'dcoilwtico',
    'sales_lag_7', 'sales_lag_14', 'sales_lag_28',
    'sales_roll_mean_7', 'sales_roll_mean_14', 'sales_roll_mean_28',
    'sales_roll_std_7', 'sales_roll_std_14', 'sales_roll_std_28',
]

target = 'sales'

print(f"\n✅ Final feature set: {len(feature_cols)} features")
print(f"Features: {feature_cols}")

---

## 6️⃣ Modeling & Evaluation

We will train **4 different models** and compare their performance to find the best forecasting approach.

| # | Model | Description |
|---|-------|-------------|
| 1 | **Baseline (Naïve)** | Predict using the last known value (lag 7) |
| 2 | **Linear Regression** | Simple linear model |
| 3 | **Random Forest** | Ensemble of decision trees |
| 4 | **LightGBM** | Gradient boosting (state-of-the-art) |

In [ ]:
# ============================================================
# 🎯 Prepare Train/Validation Split
# ============================================================
# Use time-based split: last 15 days as validation (same as test period)
val_start = df_model['date'].max() - timedelta(days=15)
train_data = df_model[df_model['date'] < val_start]
val_data = df_model[df_model['date'] >= val_start]

X_train = train_data[feature_cols]
y_train = train_data[target]
X_val = val_data[feature_cols]
y_val = val_data[target]

print(f"✅ Train/Validation split (time-based)")
print(f"   Train: {X_train.shape} | date < {val_start.date()}")
print(f"   Val:   {X_val.shape} | date >= {val_start.date()}")

# Metrics functions
def rmsle(y_true, y_pred):
    y_pred = np.clip(y_pred, 0, None)
    return np.sqrt(np.mean((np.log1p(y_pred) - np.log1p(y_true))**2))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mae(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

results = {}

In [ ]:
# ============================================================
# Model 1: Baseline (Naïve - use lag 7)
# ============================================================
baseline_pred = X_val['sales_lag_7'].values
baseline_pred = np.clip(baseline_pred, 0, None)

results['Baseline (Lag 7)'] = {
    'RMSLE': rmsle(y_val, baseline_pred),
    'RMSE': rmse(y_val, baseline_pred),
    'MAE': mae(y_val, baseline_pred)
}
print(f"✅ Model 1: Baseline")
print(f"   RMSLE: {results['Baseline (Lag 7)']['RMSLE']:.4f}")
print(f"   RMSE:  {results['Baseline (Lag 7)']['RMSE']:.2f}")
print(f"   MAE:   {results['Baseline (Lag 7)']['MAE']:.2f}")

In [ ]:
# ============================================================
# Model 2: Linear Regression
# ============================================================
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = np.clip(lr_model.predict(X_val), 0, None)

results['Linear Regression'] = {
    'RMSLE': rmsle(y_val, lr_pred),
    'RMSE': rmse(y_val, lr_pred),
    'MAE': mae(y_val, lr_pred)
}
print(f"✅ Model 2: Linear Regression")
print(f"   RMSLE: {results['Linear Regression']['RMSLE']:.4f}")
print(f"   RMSE:  {results['Linear Regression']['RMSE']:.2f}")
print(f"   MAE:   {results['Linear Regression']['MAE']:.2f}")

In [ ]:
# ============================================================
# Model 3: Random Forest (sampled for memory/speed)
# ============================================================
# Sample training data for Random Forest (3M+ rows is too large)
sample_frac = min(500_000 / len(X_train), 1.0)
X_train_sample = X_train.sample(frac=sample_frac, random_state=42)
y_train_sample = y_train.loc[X_train_sample.index]

rf_model = RandomForestRegressor(
    n_estimators=100, max_depth=15, min_samples_leaf=10,
    n_jobs=-1, random_state=42, verbose=0
)
rf_model.fit(X_train_sample, y_train_sample)
rf_pred = np.clip(rf_model.predict(X_val), 0, None)

results['Random Forest'] = {
    'RMSLE': rmsle(y_val, rf_pred),
    'RMSE': rmse(y_val, rf_pred),
    'MAE': mae(y_val, rf_pred)
}
print(f"✅ Model 3: Random Forest (trained on {len(X_train_sample):,} samples)")
print(f"   RMSLE: {results['Random Forest']['RMSLE']:.4f}")
print(f"   RMSE:  {results['Random Forest']['RMSE']:.2f}")
print(f"   MAE:   {results['Random Forest']['MAE']:.2f}")

In [ ]:
# ============================================================
# Model 4: LightGBM (handles large data efficiently)
# ============================================================
lgb_model = lgb.LGBMRegressor(
    n_estimators=500, max_depth=10, learning_rate=0.05,
    num_leaves=63, subsample=0.8, colsample_bytree=0.8,
    min_child_samples=50, reg_alpha=0.1, reg_lambda=0.1,
    n_jobs=-1, random_state=42, verbose=-1
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.log_evaluation(period=0)]
)
lgb_pred = np.clip(lgb_model.predict(X_val), 0, None)

results['LightGBM'] = {
    'RMSLE': rmsle(y_val, lgb_pred),
    'RMSE': rmse(y_val, lgb_pred),
    'MAE': mae(y_val, lgb_pred)
}
print(f"✅ Model 4: LightGBM")
print(f"   RMSLE: {results['LightGBM']['RMSLE']:.4f}")
print(f"   RMSE:  {results['LightGBM']['RMSE']:.2f}")
print(f"   MAE:   {results['LightGBM']['MAE']:.2f}")

### 📊 Model Comparison

In [ ]:
# ============================================================
# 📊 Model Comparison
# ============================================================
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('RMSLE')
print("=" * 70)
print("📊 MODEL COMPARISON (sorted by RMSLE)")
print("=" * 70)
display(results_df)

best_model_name = results_df.index[0]
print(f"\n🏆 Best Model: {best_model_name} (RMSLE: {results_df.iloc[0]['RMSLE']:.4f})")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
metrics = ['RMSLE', 'RMSE', 'MAE']
colors = [COLORS[0], COLORS[2], COLORS[4], COLORS[6]]

for ax, metric in zip(axes, metrics):
    vals = results_df[metric]
    bars = ax.bar(range(len(vals)), vals.values, color=colors[:len(vals)],
                  edgecolor='black', alpha=0.85)
    ax.set_title(f'📊 {metric}', fontweight='bold', fontsize=14)
    ax.set_xticks(range(len(vals)))
    ax.set_xticklabels(vals.index, rotation=30, ha='right', fontsize=9)
    for bar, val in zip(bars, vals.values):
        ax.text(bar.get_x() + bar.get_width()/2, val + val*0.02,
               f'{val:.4f}' if metric == 'RMSLE' else f'{val:.1f}',
               ha='center', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

### 🔍 Feature Importance (LightGBM)

In [ ]:
# Feature importance from LightGBM
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

fig, ax = plt.subplots(figsize=(12, 8))
top20 = importance.head(20)
ax.barh(range(len(top20)), top20['importance'].values, color=COLORS[4], edgecolor='black', alpha=0.85)
ax.set_yticks(range(len(top20)))
ax.set_yticklabels(top20['feature'].values)
ax.set_title('🔍 Top 20 Most Important Features (LightGBM)', fontweight='bold', fontsize=16)
ax.set_xlabel('Importance')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

print("\n📊 Top 10 Features:")
for i, row in importance.head(10).iterrows():
    print(f"   {row['feature']:30s} → {row['importance']:>8.0f}")

---

## 7️⃣ Best Model & Final Predictions

Now we use our best-performing model to generate predictions on the **test dataset** and prepare a submission file.

In [ ]:
# ============================================================
# 🏆 Generate Test Predictions
# ============================================================

# Prepare test data with same features
df_test = test.copy()

# Date features
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test['day'] = df_test['date'].dt.day
df_test['day_of_week'] = df_test['date'].dt.dayofweek
df_test['day_of_year'] = df_test['date'].dt.dayofyear
df_test['week_of_year'] = df_test['date'].dt.isocalendar().week.astype(int)
df_test['is_weekend'] = (df_test['day_of_week'] >= 5).astype(int)
df_test['is_month_start'] = df_test['date'].dt.is_month_start.astype(int)
df_test['is_month_end'] = df_test['date'].dt.is_month_end.astype(int)
df_test['is_payday'] = ((df_test['day'] == 15) | df_test['date'].dt.is_month_end).astype(int)
df_test['is_earthquake_period'] = 0

# Merge store info
df_test = df_test.merge(stores, on='store_nbr', how='left')

# Merge oil
df_test = df_test.merge(oil, on='date', how='left')
df_test['dcoilwtico'] = df_test['dcoilwtico'].interpolate().bfill().ffill()

# Holiday features
df_test['is_national_holiday'] = df_test['date'].isin(nat_holidays['date'].unique()).astype(int)
df_test['is_event'] = df_test['date'].isin(events['date'].unique()).astype(int)

# Encode categoricals (using same encoders)
df_test['family_encoded'] = le_family.transform(df_test['family'])
df_test['city_encoded'] = le_city.transform(df_test['city'])
df_test['state_encoded'] = le_state.transform(df_test['state'])
df_test['type_encoded'] = le_type.transform(df_test['type'])

# Lag features from training data (last known values)
last_known = df.groupby(['store_nbr', 'family_encoded']).tail(28).copy()
for lag in [7, 14, 28]:
    lag_vals = last_known.groupby(['store_nbr', 'family_encoded'])['sales'].apply(
        lambda x: x.iloc[-lag] if len(x) >= lag else x.mean()
    ).reset_index()
    lag_vals.columns = ['store_nbr', 'family_encoded', f'sales_lag_{lag}']
    df_test = df_test.merge(lag_vals, on=['store_nbr', 'family_encoded'], how='left')

# Rolling features from training data
for window in [7, 14, 28]:
    roll_vals = last_known.groupby(['store_nbr', 'family_encoded'])['sales'].apply(
        lambda x: x.tail(window).mean()
    ).reset_index()
    roll_vals.columns = ['store_nbr', 'family_encoded', f'sales_roll_mean_{window}']
    df_test = df_test.merge(roll_vals, on=['store_nbr', 'family_encoded'], how='left')

    roll_std_vals = last_known.groupby(['store_nbr', 'family_encoded'])['sales'].apply(
        lambda x: x.tail(window).std()
    ).reset_index()
    roll_std_vals.columns = ['store_nbr', 'family_encoded', f'sales_roll_std_{window}']
    df_test = df_test.merge(roll_std_vals, on=['store_nbr', 'family_encoded'], how='left')

# Fill any remaining NaN
df_test[feature_cols] = df_test[feature_cols].fillna(0)

print(f"✅ Test features prepared: {df_test.shape}")

In [ ]:
# ============================================================
# 🔮 Generate Predictions with Best Model (LightGBM)
# ============================================================
X_test_final = df_test[feature_cols]
test_predictions = np.clip(lgb_model.predict(X_test_final), 0, None)

# Create submission
submission_final = pd.DataFrame({
    'id': test['id'],
    'sales': test_predictions
})

# Save
submission_final.to_csv('submission.csv', index=False)
print(f"✅ Submission file saved: submission.csv")
print(f"   Shape: {submission_final.shape}")
display(submission_final.head(10))

# Summary statistics
print(f"\n📊 Prediction Summary:")
print(f"   Min:    {test_predictions.min():.2f}")
print(f"   Mean:   {test_predictions.mean():.2f}")
print(f"   Median: {np.median(test_predictions):.2f}")
print(f"   Max:    {test_predictions.max():.2f}")
print(f"   Zeros:  {(test_predictions == 0).sum()} ({(test_predictions == 0).mean()*100:.1f}%)")

In [ ]:
# ============================================================
# 📈 Visualize Predictions
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Distribution of predictions
axes[0].hist(test_predictions[test_predictions > 0], bins=100,
            color=COLORS[0], edgecolor='black', alpha=0.7)
axes[0].set_title('📊 Distribution of Predicted Sales (>0)', fontweight='bold', fontsize=14)
axes[0].set_xlabel('Predicted Sales')
axes[0].set_ylabel('Count')

# Predictions over time (total daily)
pred_daily = df_test.copy()
pred_daily['predicted_sales'] = test_predictions
daily_pred_total = pred_daily.groupby('date')['predicted_sales'].sum()
daily_pred_total.plot(ax=axes[1], color=COLORS[4], linewidth=2, marker='o', markersize=5)
axes[1].set_title('📈 Total Predicted Daily Sales (Test Period)', fontweight='bold', fontsize=14)
axes[1].set_ylabel('Total Predicted Sales')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

---

## 🎯 Conclusion

### What We Did:
1. **Loaded & explored** 7 datasets covering store sales in Ecuador (2013–2017)
2. **Deep EDA** revealed key drivers: product family, promotions, payday cycles, seasonality
3. **Feature engineering** created 30+ features from dates, stores, oil prices, holidays
4. **Model comparison** of 4 approaches showed **LightGBM** performed best
5. **Generated predictions** for the 15-day test period

### Key Takeaways for the Business:
- 📦 **GROCERY I and BEVERAGES** dominate sales — keep these well-stocked!
- 🎯 **Promotions work** — items on promotion sell significantly more
- 💰 **Pay days matter** — plan inventory for the 15th and end of month
- 📅 **Sunday is king** — highest sales day of the week
- 🎄 **December peaks** — Christmas shopping drives yearly highs
- 🛢️ Oil prices affect the economy indirectly but aren't a strong direct predictor

---

<div align="center">

### 🙏 Thank You for Reading!

**Connect with me on [LinkedIn](https://www.linkedin.com/in/sajjad-ali-shah47/)**

</div>